In [160]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
import multiprocessing
from collections import OrderedDict, defaultdict
from copy import deepcopy

TODO:
    - Add Bias term
    - Custom weights
    - Drop out connections (set weight to 0)
    - Custom activation per node

# Activation functions

In [ ]:
def leaky_relu(x):
    return F.leaky_relu(x)

def tanh(x):
    return F.Tanh(x)

def relu(x):
    return F.relu(x)

def sigmoid(x):
    return F.Sigmoid(x)

string_to_activation = {
    'leaky_relu' : leaky_relu,
    'relu' : relu,
    'sigmoid' : sigmoid,
    'tanh' : tanh
}

# Model

In [88]:
class Model(nn.Module):
    def __init__(self,layer_sizes):
        super(Model, self).__init__()
        layers = OrderedDict()
        
        previous_layer_size = layer_sizes[0]
        for idx, current_layer_size in enumerate(layer_sizes[1:]):
            layers[str(idx)] = nn.Linear(previous_layer_size, current_layer_size)
            previous_layer_size = current_layer_size
            
        self.layers = nn.Sequential(layers)
        
    def forward(self, x):
        return self.model(x)

# Genotype

In [150]:
class Genotype(object):
    def __init__(self, inputs, 
                 outputs, 
                 nonlinearities, 
                 feedforward,
                 p_add_neuron, 
                 p_add_connection, 
                 p_mutate_weight, 
                 p_reenable_connection,
                 p_disable_connection, 
                 p_reenable_parent, 
                 p_mutate_bias,
                 distance_excess_weight, 
                 distance_disjoint_weight, 
                 distance_weight):
        
        self.inputs = inputs
        self.outputs = outputs
        self.nonlinearities = nonlinearities
        self.feedforward = feedforward
        
        # Mutation Probabilities
        self.p_add_neuron = p_add_neuron
        self.p_add_connection = p_add_connection
        self.p_mutate_weight = p_mutate_weight
        self.p_reenable_connection = p_reenable_connection
        self.p_disable_connection = p_disable_connection
        self.p_reenable_parent = p_reenable_parent
        self.p_mutate_bias = p_mutate_bias
        
        # Distance weights
        self.distance_excess_weight = distance_excess_weight
        self.distance_disjoint_weight = distance_disjoint_weight
        self.distance_weight = distance_weight
        
        # Tuples of: id, non_linearity, layer
        self.neuron_genes = []
        # Tuples of: innovation number, input, output, weight, enabled
        self.connection_genes = {}
        # Hyperparameter genes
        self.hyperparameter_genes = []
        
        self._initialise_topology()
        
    def _initialise_topology(self):
        # Initialise inputs
        for i in range(self.inputs):
            self.neuron_genes.append([i * 2048, random.choice(self.nonlinearities),0])
        
        # Initialise outputs
        for i in range(self.outputs):
            self.neuron_genes.append([(self.inputs + i) * 2048, random.choice(self.nonlinearities)])
        
        # Initialiase connections
        innovation_number = 0
        for i in range(self.inputs):
            for j in range(self.outputs,self.inputs + self.outputs):
                weight = self._initialise_weight(self.inputs,self.outputs)
                self.connection_genes[(i,j)] = [innovation_number, i, j, weight ,True]
                innovation_number += 1
                
    def _initialise_weight(self, input_neurons, output_neurons):
        weight = np.random.rand()*np.sqrt(1/(input_neurons + output_neurons))
        
#     def translate_to_pytorch(self):
#         self.model = Model([self.inputs, 5, self.outputs])
        
#         print(self.model.layers[0].weight)
#         raise NotImplementedError
        
#     def get_weight_matrix(self,layer_1, layer_2):
#         raise NotImplementedError
        
    def recombinate(self, other):
        child = deepcopy(self)
        child.neuron_genes = []
        child.connection_genes = {}
        
        max_neurons = max(len(self.neuron_genes), len(other.neruon_genes))
        min_neurons = min(len(self.neuron_genes), len(other.neuron_genes))
        
        for i in range(max_neurons):
            neuron_gene = None
            if i < min_neurons:
                neuron_gene = random.choice((self.neuron_genes[i], other.neuron_genes[i]))
            else:
                try:
                    neuron_gene = self.neuron_genes[i]
                except IndexError:
                    neuron_gene = other.neuron_genes[i]
            child.neurons_genes.append(deepcopy(neuron_gene))
            
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))
        max_innovation_number = max(self_connections.keys() + other_connections.keys())
        
        for i in range(max_innovation_number + 1):
            connection_gene = None
            if i in self_connections and i in other_connections:
                connection_gene = random.choice((self_connections[i],other_connections[i]))
                enabled = self_connections[i][4] and other_connections[i][4]
            else:
                if i in self_connections:
                    connection_gene = self_connections[i]
                    enabled = connection_gene[4]
                elif i in other_connections:
                    connection_gene = other_connections[i]
                    enabled = connection_gene[4]
            if connection_gene is not None:
                child.connection_genes[(connection_gene[1],connection_gene[2])] = deepcopy(connection_gene)
                child.connection_genes[(connection_gene[1],connection_gene[2])][4] = enabled or rand() < self.p_reenable_parent

            def is_feedforward(item):
                ((fr, to), cg) = item
                return child.node_genes[fr][0] < child.node_genes[to][0]

            if self.feedforward:
                child.conn_genes = dict(filter(is_feedforward, child.conn_genes.items()))
            
        return child
        
    def mutate(self):
        # TODO: move to separate functions
        if np.random.rand() < self.p_add_neuron:
            # Choose connection to split
            split_neuron = self.connection_genes[random.choice(self.connection_genes.keys())]
            # Disable old connection
            split_neuron[4] = False
            
            input_neuron, output_neuron, weight = split_neuron[1:4]
            neuron_id = (self.neuron_genes[input_neuron][0] + self.neuron_genes[input_neuron][0]) * 0.5
            nonlinearity = random.choice(self.nonlinearities)
            layer = self.neuron_genes[input_neuron][2] + 1
            
            neuron = [neuron_id, nonlinearity, layer]
            
            neuron_id = len(self.node_genes) - 1
            
            self.neuron_genes.append(neuron)
            # 1.0 to initialise_weight?
            # TODO: get innovation number
            self.connection_genes[(input_neuron, neuron_id)] = [innovation_number, input_neuron, neuron_id, 1.0, True]
            
            self.connection_genes[(neuron_id, output_neuron)] = [innovation_number, neuron_id, output_neuron, weight, True]
        
        return self
        
    def distance(self, other):
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))
        
        all_innovations = self_connections.keys() + other_connections.keys()

# Species

In [131]:
class Species(object):
    def __init__(self, initial_member):
        self.members = [initial_member]
        self.representative = initial_member
        self.offspring = 0
        self.age = 0
        self.avg_fitness = 0.
        self.max_fitness = 0.
        self.max_fitness_previous = 0.0
        self.stagnation = 0
        self.has_best = False

# Population

In [164]:
class Population(object):
    def __init__(self, genome_factory,
                population_size,
                elitism,
                stop_when_solved,
                tournament_selection_k,
                verbose,
                max_cores,
                compatibility_threshold,
                compatibility_threshold_delta,
                target_species,
                minimum_elitism_size,
                young_age,
                young_multiplier,
                old_age,
                old_multiplier,
                stagnation_age,
                reset_innovations,
                survival):
        
        self.genome_factory = genome_factory
        self.population_size = population_size
        self.elitism = elitism
        self.stop_when_solved = stop_when_solved
        self.tournament_selection_k = tournament_selection_k
        self.verbose = verbose
        self.max_cores = max_cores
        
        cpus = multiprocessing.cpu_count()
        use_cores = min(self.max_cores, cpus-1)
        if use_cores > 1:
            self.pool = multiprocessing.pool(processing=use_cores, maxtasksperchild=5)
        else:
            self.pool = None
        
        self.compatibility_threshold = compatibility_threshold
        self.compatibility_threshold_delta = compatibility_threshold_delta
        
        self.target_species = target_species
        self.minimum_elitism_size = minimum_elitism_size
        
        self.young_age = young_age
        self.young_multiplier = young_multiplier
        self.old_age = old_age
        self.old_multiplier = old_multiplier
        
        self.stagnation_age = stagnation_age
        
        self.reset_innovations = reset_innovations
        self.survival = survival
        
    def _evaluate_all(self, population, evaluator):
        raise NotImplementedError
        
    def _reset(self):
        self.champions = []
        self.generation = 0
        self.solved_at = None
        self.stats = defaulftdict(list)
        
        self.species = []
        self.global_innovation_number = 0
        self.innovations = {}
        self.current_compatibility_threshold = self.compatibility_threshold
        
    def _evolve(self, evaluator, solution=None):
        population = list(self.population)
        
        while len(population) < self.population_size:
            individual = self.genome_factory()
            pop.append(individual)
            
        population = self._evaluate_all(population, evaluator)
        
        # Speciation
        for specie in self.species:
            # Choose random specie representative for distance comparison
            specie.representative = random.choice(specie.members)
            specie.members = []
            specie.age += 1
            
        # Add each individual to a species
        for individual in population:
            found = False
            for specie in self.species:
                if individual.distance(specie.representative) <= self.current_compatibility_threshold:
                    specie.members.append(individual)
                    found = True
                    break
                if not found:
                    s = Species(individual)
                    self.species.append(s)
    

# Test

In [167]:
inputs = 3
outputs = 4
nonlinearities = ['relu','sigmoid']
feedforward = True

p_add_node = 0.03
p_add_connection = 0.3
p_mutate_weight = 0.8
p_reenable_connection = 0.01
p_disable_connection = 0.01
p_reenable_parent=0.25
p_mutate_bias = 0.2

distance_excess_weight = 1.0
distance_disjoint_weight = 1.0
distance_weight = 0.4

genotype = Genotype(inputs, outputs, nonlinearities, feedforward, 
                    p_add_node, p_add_connection, p_mutate_weight, p_reenable_connection,
                   p_disable_connection, p_reenable_parent, p_mutate_bias,
                   distance_excess_weight, distance_disjoint_weight, distance_weight)

In [168]:
population_size = 100
elitism = True
stop_when_solved = False 
tournament_selection_k = 3 
verbose = True
max_cores = 1
compatibility_threshold = 3.0
compatibility_threshold_delta = 0.4 
target_species = 12
minimum_elitism_size = 5 
young_age = 10
young_multiplier = 1.2 
old_age = 30
old_multiplier = 0.2 
stagnation_age = 15
reset_innovations = False
survival = 0.2

genome_factory = lambda: Genotype(inputs, outputs, nonlinearities, feedforward, 
                    p_add_node, p_add_connection, p_mutate_weight, p_reenable_connection,
                   p_disable_connection, p_reenable_parent, p_mutate_bias,
                   distance_excess_weight, distance_disjoint_weight, distance_weight)

population = Population(population_size, elitism, stop_when_solved, tournament_selection_k, verbose, max_cores, compatibility_threshold, compatibility_threshold_delta, target_species, minimum_elitism_size, young_age, young_multiplier, old_age, old_multiplier, stagnation_age, reset_innovations, survival)
population._evolve()

TypeError: _evolve() missing 1 required positional argument: 'evaluator'

In [14]:
w = torch.empty(3, 5)
nn.init.xavier_uniform_(w, gain=nn.init.calculate_gain('relu'))

tensor([[-0.2442, -0.6035,  0.8889,  0.5022, -0.4859],
        [-0.2921,  1.2088,  0.4895, -0.8608,  0.1795],
        [ 1.1595, -0.7377, -0.1698,  0.0373,  1.0230]])

In [149]:
def f(inputs):
    ((fr, to), cg) =inputs
    print(fr)

conns = {}
conns[(1,5)] = [2,2,3]
dict(filter(f, conns.items()))

1


{}